In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

ur=[]
# #we are collecting url for all annime present on site
# #range is from 1 to 530 because 530 pages are there
for i in range(1,530):
    url='https://www.anime-planet.com/anime/all?page={}'.format(i)
    headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
            }
    response=requests.get(url,headers=headers)
    con=response.content
    soup=BeautifulSoup(con ,'lxml')
    for name in soup.find_all('li',class_="card"):
        for na in name.find_all('a'):
            ur.append(na.get('href'))

In [ ]:
#scrapeing proxies to avoid block by server
from bs4 import BeautifulSoup as bs
import requests
from random import choice
def get_proxies():
    proxy_url="https://github.com/clarketm/proxy-list/blob/master/proxy-list-raw.txt"
    r=requests.get(proxy_url)
    soup=bs(r.content,'html.parser').find_all('td',{"class":"blob-code blob-code-inner js-file-line"})
    proxies=[proxy.text for proxy in soup]
    return proxies
proxies=get_proxies()
url="https://www.google.com"
working=[]
def get_working_proxies():
    for i in range(400):
        proxy={"https://" :proxies[i]}
        try:
            r=requests.get(url,proxies=proxy,timeout=3)
            if r.status_code==200:
                working.append(proxy)
        except:
            pass
    return working  
proxy=get_working_proxies()

In [ ]:
col_names=['url',"Name",'Japanese_name','Type','Gener','Demographic','Score','Studio','Synopsis','Theme','Premired','Episodes','Status','Aired','Members','Brodcast','Source','Rating','Duration','Synonym','Users','Producers','Licensors','Related_Anime','Voice_Actors','img_url']
df=pd.DataFrame(columns=col_names)

In [ ]:
df['url']=ur

In [ ]:
i=0
for u in ur:
    url=u
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
        }

    proxy=random.choice(working)
    print(proxy)


    response=requests.get(url,headers=headers,proxies=proxy)
    time.sleep(2)
    con=response.content
    soup=BeautifulSoup(con ,'lxml')
    print(i)
    print(url)
    for contain in soup.find_all(class_="spaceit_pad"):
        for c in contain.find_all(class_="dark_text"):

                Japanese=[]
                if c.text=='Japanese:':
                    Japanese.append(contain.text.strip())
                    df.iloc[i,2]=Japanese[0][9:]

                type_=[]
                if c.text=="Type:":
                    if contain.find('a'):
                        for l in contain.find('a'):
                            type_.append(l.text)
                    else:
                        type_.append(c.next_sibling.strip())
                    df.iloc[i,3]=type_[0]

                Episodes=[]
                if c.text=='Episodes:':
                    Episodes.append(contain.text.strip())
                    df.iloc[i,11]=Episodes[0][11:]


                Status=[]
                if c.text=='Status:':
                    Status.append(contain.text.strip())
                    df.iloc[i,12]=Status[0][9:]


                Aired=[]
                if c.text=='Aired:':
                    Aired.append(contain.text.strip())
                    df.iloc[i,13]=Aired[0][8:]


                Premiered=[]
                if c.text=='Premiered:':
                    Premiered.append(contain.text.strip())
                    df.iloc[i,10]=Premiered[0][11:]

                Broadcast=[]
                if c.text=='Broadcast:':
                    Broadcast.append(contain.text.strip())
                    df.iloc[i,15]=Broadcast[0][12:]


                Producers=[]
                if c.text=='Producers:':
                    Producers.append(contain.text.strip())
                    df.iloc[i,21]=Producers[0][11:]

                Licensors=[]
                if c.text=='Licensors:':
                    Licensors.append(contain.text.strip())
                    df.iloc[i,22]=Licensors[0][11:]

                Studios=[]
                if c.text=='Studios:':
                    Studios.append(contain.text.strip())
                    df.iloc[i,7]=Studios[0][9:]


                Source=[]
                if c.text=='Source:':
                    Source.append(contain.text.strip())
                    df.iloc[i,16]=Source[0][9:]


                Genres=[]
                if c.text=='Genres:'or c.text=='Genre:':
                    for l in contain.find_all('a'):
                        Genres.append(l.text)
                    Genres=' , '.join(Genres)
                    df.iloc[i,4]=Genres



                Demographic=[]
                if c.text=='Demographic:' or c.text=='Demographics:':
                    for l in contain.find_all('a'):
                        Demographic.append(l.text)
                    Demographic=' , '.join(Demographic)
                    df.iloc[i,5]=Demographic

                Themes=[]
                if c.text=='Themes:' or c.text=='Theme:':
                    for l in contain.find_all('a'):
                        Themes.append(l.text)
                    Themes=' , '.join(Themes)
                    df.iloc[i,9]=Themes


                Duration=[]
                if c.text=='Duration:':
                    Duration.append(contain.text.strip())
                    df.iloc[i,18]=Duration[0][10:]


                Rating=[]
                if c.text=='Rating:':
                    Rating.append(contain.text.strip())
                    df.iloc[i,17]=Rating[0][9:]


                Synonyms=[]
                if c.text=='Synonyms:':
                    Synonyms.append(contain.text.strip())
                    df.iloc[i,19]=Synonyms[0][10:]

                Members=[]
                if c.text=='Members:':
                    Members.append(contain.text.strip())
                    df.iloc[i,14]=Members[0][10:]



    names=[]
    if soup.find_all('h1',class_='title-name h1_bold_none'):
        for name in soup.find_all('h1',class_='title-name h1_bold_none'):
            names.append(name.text.strip())
        df.iloc[i,1]=names[0]
    else:
        df.iloc[i,1]=np.nan

    synopsis=[]
    if soup.find_all(itemprop='description'):
        for desc in soup.find_all(itemprop='description'):
            synopsis.append(desc.text.strip())
        df.iloc[i,8]=synopsis[0]
    else:
        df.iloc[i,8]=np.nan

    score=[]
    if soup.find_all(itemprop='ratingValue'):
        for contain in soup.find_all(itemprop="ratingValue"):
            score.append(float(contain.text))
            print(score)
        df.iloc[i,6]=score[0]
    else:
        df.iloc[i,6]=np.nan
    users=[]
    if soup.find_all(itemprop='ratingCount'):
        for contain in soup.find_all(itemprop="ratingCount"):
            users.append(int(contain.text))
        df.iloc[i,20]=users[0]
    else:
        df.iloc[i,20]=np.nan


    anime=[]
    if soup.find_all(class_="anime_detail_related_anime"):
        for contain in soup.find_all(class_="anime_detail_related_anime"):       
            for l in contain.find_all('a'):
                anime.append(l.text)
        anime=' , '.join(anime)
        df.iloc[i,23]=anime
    else:
        df.iloc[i,23]=np.nan

    voice=[]
    if soup.find_all(class_="detail-characters-list clearfix"):
        for contain in soup.find_all(class_="detail-characters-list clearfix"):
            for con in soup.find_all(class_="va-t ar pl4 pr4"):
                if con.find_all('a'):
                    a=con.find_all('a')
                    for x in a:
                        voice.append(x.text.strip())
                else:
                    voice.append(np.nan)
        voice=' ; '.join(voice)
        df.iloc[i,24]=voice




    img=[]
    for name in soup.find_all(style="text-align: center;"):
        for na in name.find_all('img'):
            if name.find_all('img'):
                img.append(na.get('data-src'))
                df.iloc[i,25]=img[0]

    i=i+1